In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_data = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
train_data.head()

In [ ]:
lst1 = list(train_data['Dates'])
for i in range(len(lst1)):
    lst1[i]=int(lst1[i][5:7])
train_data['Month']=lst1

lst1 = list(test_data['Dates'])
for i in range(len(lst1)):
    lst1[i]=int(lst1[i][5:7])
test_data['Month']=lst1

In [ ]:
train_data['Month'].replace([1,2,3,4,5,6,7,8,9,10,11,12], ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], inplace=True)

test_data['Month'].replace([1,2,3,4,5,6,7,8,9,10,11,12], ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], inplace=True)

In [ ]:
lst3 = list(train_data['Dates'])
for i in range(len(lst3)):
    lst3[i]=lst3[i][11:13]
train_data['TimeInHr']=lst3

lst3 = list(test_data['Dates'])
for i in range(len(lst3)):
    lst3[i]=lst3[i][11:13]
test_data['TimeInHr']=lst3

In [ ]:
lst4 = list(train_data['Dates'])
for i in range(len(lst4)):
    lst4[i]=int(lst4[i][:4])
train_data['Year']=lst4

lst4 = list(test_data['Dates'])
for i in range(len(lst4)):
    lst4[i]=int(lst4[i][:4])
test_data['Year']=lst4

In [ ]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['DayOfWeek'])], axis=1)
train_data.drop('DayOfWeek', axis=1, inplace=True)

test_data = pd.concat([test_data, pd.get_dummies(test_data['DayOfWeek'])], axis=1)
test_data.drop('DayOfWeek', axis=1, inplace=True)

In [ ]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['PdDistrict'])], axis=1)
train_data.drop('PdDistrict', axis=1, inplace=True)

test_data = pd.concat([test_data, pd.get_dummies(test_data['PdDistrict'])], axis=1)
test_data.drop('PdDistrict', axis=1, inplace=True)

In [ ]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['Month'])], axis=1)
train_data.drop('Month', axis=1, inplace=True)

test_data = pd.concat([test_data, pd.get_dummies(test_data['Month'])], axis=1)
test_data.drop('Month', axis=1, inplace=True)

In [ ]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['TimeInHr'])], axis=1)
train_data.drop('TimeInHr', axis=1, inplace=True)

test_data = pd.concat([test_data, pd.get_dummies(test_data['TimeInHr'])], axis=1)
test_data.drop('TimeInHr', axis=1, inplace=True)

In [ ]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['Year'])], axis=1)
train_data.drop('Year', axis=1, inplace=True)

test_data = pd.concat([test_data, pd.get_dummies(test_data['Year'])], axis=1)
test_data.drop('Year', axis=1, inplace=True)

In [ ]:
train_data.drop(['Address', 'Descript', 'Resolution', 'Dates'], axis=1, inplace=True)

test_data.drop(['Address', 'Dates'], axis=1, inplace=True)

In [ ]:
X = train_data.drop(['Category'],axis=1)
y = pd.get_dummies(train_data['Category'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from tensorflow import keras

In [ ]:
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Dense(128, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(39, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
train = model.fit(x=X_train, y=y_train,
                  batch_size=16,
                  epochs=8,
                  verbose=2,
                 validation_data=(X_test, y_test))

In [ ]:
test=test_data.drop(['Id'],axis=1)
test=test.astype(float)
test.info()

In [ ]:
pred=model.predict(test)
print(pred)

In [ ]:
pred_final=[np.argmax(i) for i in pred]

In [ ]:
m_reshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == m_reshape), dtype='int32')
pred_final

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
col_names=list(sample_submission.columns)
col_names.remove('Id')
print(col_names)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_data['Id']
for i , entry in enumerate(col_names):
    submission[entry] = pred_final[:,i]

In [ ]:
submission.to_csv('../working/submission.csv', index=False)